In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


In [ ]:
!pip install opendatasets
import opendatasets as od

In [ ]:
{"username":"jhonalexandera","key":"9bde6e3f5f31a568e1ba313650d39fec"}

In [ ]:
dataset_link="https://www.kaggle.com/competitions/udea-ai-4-eng-20251-pruebas-saber-pro-colombia"
od.download(dataset_link)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jhonalexandera
Your Kaggle Key: ··········


100%|██████████| 29.9M/29.9M [00:00<00:00, 743MB/s]


Extracting archive ./udea-ai-4-eng-20251-pruebas-saber-pro-colombia/udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip to ./udea-ai-4-eng-20251-pruebas-saber-pro-colombia


In [ ]:
import os
os.chdir("udea-ai-4-eng-20251-pruebas-saber-pro-colombia")
os.listdir()

['submission_example.csv', 'train.csv', 'test.csv']

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


In [ ]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)


shape of loaded dataframe (692500, 21)


In [ ]:
z.head(3)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264


In [ ]:
z.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              692500 non-null  int64  
 1   PERIODO                         692500 non-null  int64  
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object 
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object 
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object 
 7   FAMI_TIENEINTERNET              665871 non-null  object 
 8   FAMI_EDUCACIONPADRE             669322 non-null  object 
 9   FAMI_TIENELAVADORA              652727 non-null  object 
 10  FAMI_TIENEAUTOMOVIL             648877 non-null  object 
 11  ESTU_PRIVADO_LIBERTAD           692500 non-null  object 
 12  ESTU_PAGOMATRICU

# Preprocesado y limpieza

In [ ]:
# Copia del dataframe original
df2 = z.copy()

In [ ]:
# Lista completa de columnas a eliminar (exceptuando 'RENDIMIENTO_GLOBAL')
cols_to_drop = [
    'PERIODO',
    'ESTU_VALORMATRICULAUNIVERSIDAD',
    'ESTU_HORASSEMANATRABAJA',
    'FAMI_TIENEINTERNET',
    'FAMI_EDUCACIONPADRE',
    'FAMI_TIENELAVADORA',
    'FAMI_TIENEAUTOMOVIL',
    'ESTU_PRIVADO_LIBERTAD',
    'ESTU_PAGOMATRICULAPROPIO',
    'FAMI_TIENECOMPUTADOR',
    'FAMI_TIENEINTERNET.1',
    'FAMI_EDUCACIONMADRE'
]

# Aplicamos el cambio
df2 = df2.drop(columns=cols_to_drop)


In [ ]:
# 2. Llenar valores nulos

for col in df2.select_dtypes(include='number').columns:
    df2[col] = df2[col].fillna(df2[col].median())

for col in df2.select_dtypes(include='object').columns:
    df2[col] = df2[col].fillna(df2[col].mode()[0])

In [ ]:
# 4. Mapeo informativo basado en el contexto colombiano

departamento_map = {
    'BOGOTÁ': 3, 'ANTIOQUIA': 3, 'VALLE': 3,
    'SANTANDER': 2.5, 'ATLANTICO': 2.5, 'CUNDINAMARCA': 2.5, 'BOLIVAR': 2.5, 'CALDAS': 2.5,
    'TOLIMA': 2, 'HUILA': 2, 'RISARALDA': 2, 'QUINDIO': 2, 'BOYACA': 2, 'META': 2, 'NARIÑO': 2, 'CAUCA': 2,
    'CORDOBA': 1.5, 'NORTE SANTANDER': 1.5, 'SUCRE': 1.5, 'MAGDALENA': 1.5, 'CESAR': 1.5,
    'CHOCO': 1, 'PUTUMAYO': 1, 'GUAVIARE': 1, 'VAUPES': 1, 'AMAZONAS': 1, 'SAN ANDRES': 1,
    'ARAUCA': 1, 'CAQUETA': 1, 'CASANARE': 1, 'LA GUAJIRA': 1,
}

df2['ESTU_PRGM_DEPARTAMENTO'] = df2['ESTU_PRGM_DEPARTAMENTO'].map(departamento_map)



In [ ]:
# 6. Cambiamos el tipo de estrato a un entero

df2['FAMI_ESTRATOVIVIENDA'] = df2['FAMI_ESTRATOVIVIENDA'].replace('Sin Estrato', 'Estrato 0')
df2['FAMI_ESTRATOVIVIENDA'] = df2['FAMI_ESTRATOVIVIENDA'].str.extract('Estrato (\d)').astype(int)


In [ ]:
# 8. Mapeo informativo basado en el contexto colombiano

rendimiento_map = {
    'bajo': 0,
    'medio-bajo': 1,
    'medio-alto': 2,
    'alto': 3
}
df2['RENDIMIENTO_GLOBAL'] = df2['RENDIMIENTO_GLOBAL'].map(rendimiento_map)


In [ ]:
# 9. Ejemplo de mapeo por áreas (ajusta según tu criterio)

def asignar_puntaje(programa):
    if 'INGENIERIA' in programa:
        return 4.5  # Puntaje alto para ingenierías
    elif 'MEDICINA' in programa or 'ODONTOLOGIA' in programa:
        return 4.7  # Puntaje muy alto para medicina
    elif 'ADMINISTRACION' in programa:
        return 4.0  # Puntaje medio para administración
    elif 'LICENCIATURA' in programa:
        return 3.8  # Puntaje para licenciaturas
    else:
        return 3.5  # Puntaje por defecto

df2['ESTU_PRGM_ACADEMICO'] = df2['ESTU_PRGM_ACADEMICO'].apply(asignar_puntaje)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# 10. Seleccionar columnas tipo object (categóricas)
label_cols = df2.select_dtypes(include='object').columns
if 'RENDIMIENTO_GLOBAL' in label_cols:
    label_cols = label_cols.drop('RENDIMIENTO_GLOBAL')

# 11. Codificar columnas categóricas
label_encoders = {}
for col in label_cols:
    le = LabelEncoder()
    df2[col] = le.fit_transform(df2[col].astype(str))
    label_encoders[col] = le

# 12. (Solo si no se ha hecho antes) Codificar la variable objetivo
if df2['RENDIMIENTO_GLOBAL'].dtype == 'object':
    target_encoder = LabelEncoder()
    df2['RENDIMIENTO_GLOBAL'] = target_encoder.fit_transform(df2['RENDIMIENTO_GLOBAL'])

# 13. Separar X e y
X = df2.drop(['ID', 'RENDIMIENTO_GLOBAL'], axis=1)
y = df2['RENDIMIENTO_GLOBAL']

print("Preprocesamiento completo. Shape X:", X.shape)


Preprocesamiento completo. Shape X: (692500, 7)


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID                      692500 non-null  int64  
 1   ESTU_PRGM_ACADEMICO     692500 non-null  float64
 2   ESTU_PRGM_DEPARTAMENTO  692500 non-null  float64
 3   FAMI_ESTRATOVIVIENDA    692500 non-null  int64  
 4   RENDIMIENTO_GLOBAL      692500 non-null  int64  
 5   coef_1                  692500 non-null  float64
 6   coef_2                  692500 non-null  float64
 7   coef_3                  692500 non-null  float64
 8   coef_4                  692500 non-null  float64
dtypes: float64(6), int64(3)
memory usage: 47.6 MB


In [ ]:
from sklearn.model_selection import train_test_split

Xtr, Xts, ytr, yts = train_test_split(X, y, train_size=0.8, random_state=42)

print("Train:", Xtr.shape, ytr.shape)
print("Test:", Xts.shape, yts.shape)


Train: (554000, 7) (554000,)
Test: (138500, 7) (138500,)


#Implementacion del modelo lineal

In [ ]:
from sklearn.linear_model import LogisticRegression

# Puedes ajustar max_iter si da error de convergencia
lr = LogisticRegression(max_iter=200)
lr.fit(Xtr, ytr)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

# Predicciones
preds_tr = lr.predict(Xtr)
preds_ts = lr.predict(Xts)

# Accuracy
acc_train = np.mean(preds_tr == ytr)
acc_test = np.mean(preds_ts == yts)

print("Accuracy - train:", acc_train)
print("Accuracy - test:", acc_test)


Accuracy - train: 0.33837184115523466
Accuracy - test: 0.33966064981949456
